In [ ]:
!pip install gensim
!pip install textblob

     |████████████████████████████████| 24.1 MB 26.1 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.svm import SVC
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
import nltk
from textblob import TextBlob
from langdetect import detect
import requests

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
random_train_data = pd.read_csv("/datasets/datd/DATD+Rand_training.csv")
train_data = pd.read_csv("/datasets/datd/DATD_training.csv") #already annotated. Will be used for new models
test_data = pd.read_csv("/datasets/datd/DATD_and_DATD+Rand_test.csv")
twint_depressed_tweets = pd.read_csv('twint_depressed_tweets.csv') #Need 1500 of these tweets for annotation
regular_tweets = pd.read_csv('regular_tweets.csv')

In [ ]:
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

twint_depressed_tweets['sentiment'] = twint_depressed_tweets['tweet'].apply(get_polarity)

In [ ]:
depressive_annotated = pd.read_csv("depressive_annotated.csv")

In [ ]:
depressive_annotated.head()

,tweet,sentiment,annotation
0,I sont know who this is. I should feel awful....,-1.0,1.0
1,Why does depression always hit at the worst ti...,-1.0,1.0
2,i see @thegreatkhalid on friday and i already ...,-1.0,1.0
3,what's the worst depression meal you've ever m...,-1.0,1.0
4,"dear ppl who r trying 2 save me, srsly, think ...",-1.0,1.0


In [ ]:
depressive_annotated.dropna(subset=['tweet'], inplace=True)

In [ ]:
def detectEn(tweet):
    if len(tweet) > 10:
        return detect(tweet)
    else:
        return True

In [ ]:
depressive_annotated_en = depressive_annotated[depressive_annotated['tweet'].apply(detectEn).eq('en')]

In [ ]:
depressive_annotated_en

,tweet,sentiment,annotation
0,I sont know who this is. I should feel awful....,-1.000000,1.0
1,Why does depression always hit at the worst ti...,-1.000000,1.0
2,i see @thegreatkhalid on friday and i already ...,-1.000000,1.0
3,what's the worst depression meal you've ever m...,-1.000000,1.0
4,"dear ppl who r trying 2 save me, srsly, think ...",-1.000000,1.0
...,...,...,...
3094,Republicans take a shit and expect the Democra...,0.033333,0.0
3095,"Social Media, But Not Video Games, Linked To D...",0.033333,0.0
3096,@PHOReverIM Yyyeeesss!!! Never be embarrassed ...,0.033333,0.0
3097,red velvet be like: fuck depression 😠 love ha...,0.033333,0.0


In [ ]:
depressive_annotated_en['annotation'].value_counts()

1.0    1969
0.0     665
Name: annotation, dtype: int64

In [ ]:
regular_tweets_sample = regular_tweets.sample(2000)

In [ ]:
#Regular Tweets
# regular_tweets = regular_tweets[['message', 'label']]
regular_tweets_sample = regular_tweets_sample[['message', 'label']]
# regular_tweets['label'] = 'OTHER'
regular_tweets_sample['label'] = 0
# regular_tweets = regular_tweets.rename(columns={"message": "text"})
regular_tweets_sample = regular_tweets_sample.rename(columns={"message": "text"})

#Depression Tweets
# twint_depressed_tweets = twint_depressed_tweets[['tweet']]
# twint_depressed_tweets['label'] = 'MENTAL_HEALTH'
# twint_depressed_tweets = twint_depressed_tweets.rename(columns={'tweet': 'text'})
depressive_annotated_en = depressive_annotated_en[['tweet', 'annotation']]
depressive_annotated_en = depressive_annotated_en.rename(columns={'tweet': 'text', 'annotation': 'label'})

In [ ]:
label_map = {'MENTAL_HEALTH':1,"OTHER":0}

train_data['label'] = [label_map[x] for x in train_data['label']]

In [ ]:
train_data

,text,label
0,@user Me talking to my antidepressants,1
1,going to attempt at least to change this sour ...,1
2,flighty sag placements make me anxioussss,0
3,I'm going to buy cream cheese after work and s...,1
4,My anxiety and stress levels for today are act...,1
...,...,...
895,Libra culture is using the same nail polish co...,0
896,"anyways, my last year of college starts today ...",0
897,"'Dad.. I'm depressed.' 'Hi depressed, I'm Dad.'",0
898,respiratory depression be my god,0


In [ ]:
# frames = [regular_tweets, twint_depressed_tweets, train_data]
frames = [regular_tweets_sample, depressive_annotated_en, train_data]

result = pd.concat(frames)
result

,text,label
93,"@MELindsey lol, he has no shame",0.0
5416,@do0dlebugdebz haha. LOL I'm ok with the girl ...,0.0
298,@RachaelPhillips we got beaten... but less tha...,0.0
1661,Long weekends are perfect to work on projects,0.0
2520,@VanNessVanWu ??!! can i ask for a prayer requ...,0.0
...,...,...
895,Libra culture is using the same nail polish co...,0.0
896,"anyways, my last year of college starts today ...",0.0
897,"'Dad.. I'm depressed.' 'Hi depressed, I'm Dad.'",0.0
898,respiratory depression be my god,0.0


## Preprocessing

In [ ]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

In [ ]:
from gensim.parsing.porter import PorterStemmer
p = PorterStemmer()

In [ ]:
tweet = 'this is a depressive tweet'
processed_punc = re.sub(r'[^@a-zA-Z ]', '', tweet)
processed = re.sub('https:\S+|@\S+|\b\w{,1}\b', '', processed_punc)
tokens = word_tokenize(processed)
tokens_sw = [token for token in tokens if not token in stopwords.words()]
sentence = ' '.join(tokens_sw)
stemmed_sentence = p.stem_sentence(sentence)
x_train_tokenized = vectorizer.transform([stemmed_sentence])

In [ ]:
def preprocessing(tweets_array):
    
    """
    Take in an array of tweets, and return the processed tweets.
    remove stopwords, usernames, punctuation, and words with length of 1
    """
    
    processed_array = []
    
    for tweet in tqdm(tweets_array):
        processed_punc = re.sub('[^@a-zA-Z ]', '', tweet)  # remove punctuation
        processed = re.sub('https\S+|@\S+|\b\w{,1}\b', '', processed_punc)  # remove https, usernames, len(word) < 1
        # keep only alphabets, whitespaces, and @ (to remove usernames)
        # processed = re.sub('[^@a-zA-Z ]', '', tweet)
        # #print(processed)
        # no_urls = remove_URL(processed)
        # #print(no_urls)

        # # usernames (anything that begins with "@")
        # no_usernames = " ".join([x for x in no_urls.split() if x[0] != '@'])
        # no_usernames = " ".join([x for x in processed.split() if x[0] != '@'])
        #print(no_usernames)

        words = word_tokenize(processed)

        # stopwords
        tokens_without_sw = [word for word in words if not word in stopwords.words()]
        sentence = ' '.join(tokens_without_sw)
        stemmed_sentence = p.stem_sentence(sentence)
        
        # remove words with length of 1
        processed_array.append(stemmed_sentence)
    
    return processed_array

In [ ]:
tweet = "@daniesq I know you're always up early but why https://abc.com"
processed_punc = re.sub('[^@a-zA-Z ]', '', tweet)  # remove punctuation
processed = re.sub('https\S+|@\S+|\b\w{,1}\b', '', processed_punc)
stemmed_sentence = p.stem_sentence(processed)
words = word_tokenize(stemmed_sentence)
tokens_without_sw = [word for word in words if not word in stopwords.words()]

In [ ]:
result['processed'] = preprocessing(result['text'])

100%|██████████| 5534/5534 [12:12<00:00,  7.55it/s]


In [ ]:
pd.DataFrame({"tweet": "text"}, index=[0])['tweet'].to_numpy()

array(['text'], dtype=object)

In [ ]:
np.asarray('text')

array('text', dtype='<U4')

In [ ]:
result['processed']

93                                              lol shame
5416    haha lol im ok girl tho nitesss im head sabah ...
298                    got beaten less get beaten without
1661                    long weekend perfect work project
2520                                 ask prayer request v
                              ...                        
895     libra cultur us nail polish color salon visit ...
896      anywai last year colleg start todai whew anxieti
897                      dad im depress hi depress im dad
898                               respiratori depress god
899                       deactiv main feel invis depress
Name: processed, Length: 5534, dtype: object

In [ ]:
result.dropna(subset=['processed'], inplace=True)

In [ ]:
result['processed'].value_counts()

grime claim remov part eyebal cure season depress                              38
depress                                                                         8
thank                                                                           8
                                                                                7
depress bitch                                                                   7
                                                                               ..
arlington spelt wrong give anxieti                                              1
i think i need quottweet peopl watchingquot seem like work lobbi air condit     1
either job titl would look nice resum actual i salari titl                      1
bedd peace                                                                      1
honestli lucki best mate i go depress listen i need                             1
Name: processed, Length: 5422, dtype: int64

In [ ]:
result

,text,label,processed
4462,"@xwidep that's exactly what I had for dinner, ...",0.0,that exactli i dinner except i onion bhaji nom
2347,tired of wondering if you will ever come aroun...,0.0,tire wonder ever around cali miss babi simpli ...
4905,@sterrarim Potato head!?? hahaha *ahem* sorry ...,0.0,potato head hahaha ahem sorri clearli rice
5786,@CrysAnGeL77 wow I didn't know lol yes I'm out...,0.0,wow i didnt know lol ye im alreadi well gnite tty
2624,#FollowFriday @catdog03 @babyrabies @NoPasaNad...,0.0,followfridai make laugh
...,...,...,...
3094,Republicans take a shit and expect the Democra...,0.0,republican shit expect democrat clean upeveri ...
3095,"Social Media, But Not Video Games, Linked To D...",0.0,social media but not video game link to depres...
3096,@PHOReverIM Yyyeeesss!!! Never be embarrassed ...,0.0,yyyeeesss never embarrass your inspir million ...
3097,red velvet be like: fuck depression 😠 love ha...,0.0,red velvet like fuck depress love hapi pictwit...


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(result, test_size=0.2, random_state=7)

In [ ]:
train_data.head(2)

,text,label,processed
599,Let me keep it 💯 most of my depression will ca...,1.0,let keep depress calm i monei ass
412,What I like to do when depression hits drink.....,1.0,what i like depress hit drinkdrink hard


In [ ]:
test_data.head(2)

,text,label,processed
1,Why does depression always hit at the worst ti...,1.0,why depress alwai hit worst time
7145,"@Monica2112 Oh don't worry, I don't mind if yo...",0.0,oh dont worri i dont mind im happi meet


In [ ]:
x_train = np.asarray(train_data['processed'])
y_train = np.asarray(train_data['label'])

# label_map = {cat: index for index, cat in enumerate(np.unique(y_train))}
# y_prep_train = np.asarray([label_map[l] for l in y_train])

# label_map

In [ ]:
# test_data['processed'] = preprocessing(test_data['text']) #not needed for the new datasets

In [ ]:
# test_data

In [ ]:
x_test = np.asarray(test_data['processed'])
y_test = np.asarray(test_data['label'])

# y_prep_test = np.asarray([label_map[l] for l in y_test])

## TF - IDF

In [ ]:
len(x_train)

3711

In [ ]:
x_train

array(['listen neyo closer still good song',
       'just experienc world largest hershei bar',
       'lack materialist thing doesnt caus depress believ ill time show',
       ..., 'for httpblipfmjbp', 'welcom twitter', 'doe depress'],
      dtype=object)

In [ ]:
import pickle

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(x_train)

In [ ]:
pickle.dump(vectorizer, open("vectorizer_DATD.pickle","wb"))

In [ ]:
test_tfidf = vectorizer.transform(x_test)

In [ ]:
x_test

array(['am i think time bed good night twitt',
       'everyon prealbum depress need watch taylor netflix cheer netflix',
       'what expect marri someon depress tho cours im cold sometim i dunno i forgiv cheater',
       ..., 'dark deep depress thank brain',
       'i put ibuprofenacetaminophen regimen teeth h e l l head last dai i recal someth nsaid exacerb depress im troubl find articlestudi i read comprehend can anyon help',
       'just heard bootleg remix quotif seek amyquot kiss fm chicago'],
      dtype=object)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
# train_model = model.fit(train_tfidf, y_prep_train)
train_model = model.fit(train_tfidf, y_train)
predictions = model.predict(test_tfidf)
# print('The Accuracy is ', f"{accuracy_score(predictions,y_prep_test):.2%}")
print('The Accuracy is ', f"{accuracy_score(predictions,y_test):.2%}")

The Accuracy is  80.58%


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# train_model = model.fit(train_tfidf, y_prep_train)
train_model = model.fit(train_tfidf, y_train)
predictions = model.predict(test_tfidf)
# print('The Accuracy is ', f"{accuracy_score(predictions,y_prep_test):.2%}")
print('The Accuracy is ', f"{accuracy_score(predictions,y_test):.2%}")

The Accuracy is  81.21%


In [ ]:
model.predict(x_train_tokenized)

array([0.])

In [ ]:
!pip install xgboost

     |████████████████████████████████| 173.5 MB 13 kB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(train_tfidf, y_train)

predictions = xgb_model.predict(test_tfidf)
# print('The Accuracy is ', f"{accuracy_score(predictions,y_prep_test):.2%}")
print('The Accuracy is ', f"{accuracy_score(predictions,y_test):.2%}")

/root/venv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[23:56:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The Accuracy is  80.94%


In [ ]:
predictions

array([1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 1.

In [ ]:
A = pd.DataFrame({'tweet':x_test, 'Label': predictions})
A.head()

,tweet,Label
0,why depress alwai hit worst time,1.0
1,oh dont worri i dont mind im happi meet,0.0
2,depress hurt stai strong know sorri your depre...,1.0
3,i feel depress come,1.0
4,just holler habari relat question someon hear,0.0


In [ ]:
x_test

array(['why depress alwai hit worst time',
       'oh dont worri i dont mind im happi meet',
       'depress hurt stai strong know sorri your depress dont live long prosper',
       'i feel depress come',
       'just holler habari relat question someon hear',
       'depress eat soul',
       'a movi lana parilla ami adam christina hendrick bryce dalla howard sophia bush would awai everi ounc depress',
       'that exampl i guess usual term us peopl go deep depress also wait mix togeth',
       'get bedmayb', 'i enough bodi hair qualifi',
       'thi cookbook sai recip depress turn centuri similar recip stori throughout recip collect',
       'bahah he stalker got paid go home sweet job dai good beat though go mexican',
       'no wai got up at mayb damn i like kid christma i know bad',
       'gon drive drill back',
       'on biggest fear move countri i dont leav jamaica i comfort go elsewher worst depress alex',
       'i get sad erika move sundai mooncak decid first time bite wris

In [ ]:
test_tfidf.shape

(927, 7372)

In [ ]:
xgb_model.predict(x_train_tokenized)

array([0.])

## Train Word Embeddings

In [ ]:
from gensim.utils import simple_preprocess

x_train_tokenized = [simple_preprocess(sentence, deacc=True) for sentence in x_train] 
print(x_train_tokenized[:3])

[['statesboro', 'fun', 'see', 'greenjacket', 'augusta', 'tonight'], ['love', 'pleas', 'visit', 'philippin'], ['depress', 'anxieti', 'patient', 'lung', 'cancer', 'lungcanc']]


In [ ]:
# from gensim.parsing.porter import PorterStemmer
# porter_stemmer = PorterStemmer()
# # Get the stemmed_tokens
# x_train_stemmed = [[porter_stemmer.stem(word) for word in tokens] for tokens in x_train_tokenized ]
# print(x_train_stemmed[:3])

In [ ]:
from gensim.models import Word2Vec
import time

size = 300
min_count = 1

start_time = time.time()
stemmed_tokens = x_train_tokenized
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, vector_size = size, window = 4)
print("Time taken to train word2vec model: " + str(time.time() - start_time))

Time taken to train word2vec model: 0.548851490020752


In [ ]:
embeddings_train = []

for sentence_tokens in x_train_tokenized:
    if len(sentence_tokens) != 0:
        num_rep_1 = [w2v_model.wv[token] for token in sentence_tokens]

        num_rep = np.mean(num_rep_1, axis = 0).tolist()
    else:
        num_rep = [0] * size
    embeddings_train.append(num_rep)

In [ ]:
min([len(x) for x in embeddings_train])

300

In [ ]:
# x_train_tokenized[]

In [ ]:
x_test_tokenized = [simple_preprocess(sentence, deacc=True) for sentence in x_test] 
print(x_test_tokenized[:3])

[['good', 'thing', 'amp', 'job'], ['sinc', 'doe', 'get', 'eyebal', 'surgeri', 'help', 'depress'], ['hang', 'matter', 'scene', 'gai', 'straight', 'woman', 'educ', 'ignor', 'bad', 'bougi', 'much', 'know', 'fear', 'onli', 'fear', 'accept', 'defeat', 'fear', 'anxieti', 'overcom', 'depress']]


In [ ]:
train_words = list(w2v_model.wv.key_to_index.keys())

In [ ]:
embeddings_test = []

for sentence_tokens in x_test_tokenized:
    count += 1
    if len(sentence_tokens) != 0:
        words = filter(lambda x: x in train_words, sentence_tokens)
        num_rep_1 = [w2v_model.wv[token] for token in words]
        
        if len(num_rep_1) == 0:
            num_rep = [0] * size
        else:
            num_rep = np.mean(num_rep_1, axis = 0).tolist()
    else:
        num_rep = [0] * size
    embeddings_test.append(num_rep)

NameError: name 'x_test_tokenized' is not defined

## Model

In [ ]:
start = time.time() 

svm_classifier = SVC()
# svm_classifier.fit(embeddings_train, y_prep_train)
svm_classifier.fit(embeddings_train, y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 1.86 seconds


In [ ]:
predictions = svm_classifier.predict(embeddings_test)

print('The Accuracy is ', f"{accuracy_score(predictions,y_test):.2%}")

NameError: name 'svm_classifier' is not defined

In [ ]:
xgb_model_1 = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
# xgb_model.fit(train_tfidf, y_prep_train)
xgb_model_1.fit(embeddings_train, y_train)

predictions = xgb_model_1.predict(embeddings_test)
# print('The Accuracy is ', f"{accuracy_score(predictions,y_prep_test):.2%}")
print('The Accuracy is ', f"{accuracy_score(predictions,y_test):.2%}")

NameError: name 'xgb' is not defined

In [ ]:

pickle.dump(xgb_model, open('depressive_xgb_DATD.pickle', 'wb'))

In [ ]:
vectorizer = pickle.load(open("vectorizer.pickle", "rb"))

In [ ]:
import pickle

In [ ]:
depressive = pickle.load(open('depressive_xgb.pickle', 'rb'))

In [ ]:
tweet = 'this is a depressive tweet'
processed_punc = re.sub(r'[^@a-zA-Z ]', '', tweet)
processed = re.sub('https:\S+|@\S+|\b\w{,1}\b', '', processed_punc)
tokens = word_tokenize(processed)
tokens_sw = [token for token in tokens if not token in stopwords.words()]
sentence = ' '.join(tokens_sw)
stemmed_sentence = p.stem_sentence(sentence)
x_train_tokenized = vectorizer.transform([stemmed_sentence])

prediction = depressive.predict_proba(x_train_tokenized)
prediction

NameError: name 're' is not defined

In [ ]:
prediction[0][1]

0.8106259

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c5daf6a9-77df-4bf6-8e7c-d9e1b1f200d4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>